In [67]:
import numpy as np
import matplotlib.pyplot as plt
import re
import time

In [68]:
from HTMLParser import HTMLParser

def html_decode(data):
    
    processed_data = []
    
    h = HTMLParser()
    for line in data:
        processed_data.append(h.unescape(line))
    
    return processed_data

In [69]:
class TweetConstructor:
    
    def __init__(self, max_tweet_len):
        self.max_tweet_len = max_tweet_len
        
    def begin(self):
        self.tweets = []
        self.cutmarks = []
        self.current = []
        self.current_len = 0
        
    def end(self):
        self._push_tweet()

    def _push_tweet(self, cut=False):
        
        if self.current_len > 0:
            self.tweets.append(self.current)
            self.cutmarks.append(cut)
            self.current = []
            self.current_len = 0
            
    def _add_current(self, line):
        self.current.append(line)
        self.current_len += len(line)
    
    def _clean_line(self, line):
        line = line.strip()
        
        rt = False
        cut = False
        
        if len(line) > 2 and line[:2] == u'RT':
            rt = True
            line = line[2:]
            
        if len(line) > 1 and line[-1] == u'\u2026':
            cut = True
            line = line[:-1]
            
        line = line.strip()
        return (line, rt, cut)
    
    
    def add_line(self, line):
        
        line, rt, cut = self._clean_line(line)
        
        line = line.strip()
        
        if rt:
            self._push_tweet()
        
        self._add_current(line)
        
        if cut:
            self._push_tweet(cut=True)
                
def reconstruct_tweets(data):
    tc = TweetConstructor(140)

    tc.begin()
    
    for line in data:
        tc.add_line(line)
        
    tc.end()
    
    return tc.tweets, tc.cutmarks

def merge_tweets(tweets):
    data = []
    for tweet in tweets:
        data.append((u"\n").join(tweet))
    return data


# Merge the tweets based on a heuristic (dynamic programming approach)
def merge_tweets_dp(tweets):
    
    max_len = 141
    merged_tweets = []
    for tweet in tweets:
        
        W = np.zeros((len(tweet), len(tweet)), dtype=float)
        I = -np.ones((len(tweet), len(tweet)), dtype=int)
        L = np.array([len(w) for w in tweet]) + 1
        C = np.hstack((0, np.cumsum(L)))
        C[-1] -= 1
        
        n = np.ceil(float(C[-1]) / max_len)
                
        for d in range(0, len(tweet)):
            for i in range(len(tweet) - d):
                if C[i + d + 1] - C[i] <= max_len:
                    W[i, i + d] = np.abs(C[-1] / n - (C[i + d + 1] - C[i]))
                else:
                    I[i, i + d] = np.argmin(W[i, i:(i+d)] + W[(i+1):(i+d+1), i + d]) + 1
                    W[i, i + d] = np.min(W[i, i:(i+d)] + W[(i+1):(i+d+1), i + d])
        
        merged_tweet = []
        d = 0
        while True:
            i = I[d, -1]
            if i == -1:
                break
            d += i
            merged_tweet.append(u'\n'.join(tweet[:i]))
            tweet = tweet[i:]
        merged_tweet.append(u'\n'.join(tweet))
        
        merged_tweets.extend(merged_tweet)
    
    return merged_tweets

def remove_cut_words(tweets, cutmarks):
    cleaned_tweets = []
    for tweet, cut in zip(tweets, cutmarks):
        if cut:
            id = tweet.rfind(' ')
            tweet = tweet[:id]
        cleaned_tweets.append(tweet)
    return cleaned_tweets
        

In [70]:
def link_hook(string):
    return "<LINK:" + string.group(0) + ">"

def emoji_hook(string):         
    return '<EMOJIS:' + string.group().encode('unicode-escape') + '>'

def name_hook(string):         
    return '<NAME:' + string.group(1) + '>'

def hashtag_hook(string):         
    return '<HASH:' + string + '>'

def process_links(lines, remove=False):
    
    process = '' if remove else link_hook
    
    cleaned_lines = []
    for line in lines:
        cleaned_line = re.sub('(https?://\S*)', process, line)
        cleaned_lines.append(cleaned_line)
    return cleaned_lines

def process_names(lines, remove=False):
    
    process = '' if remove else name_hook
    
    cleaned_lines = []
    for line in lines:
        cleaned_line = re.sub('@(\w+):?', process, line)
        cleaned_lines.append(cleaned_line)
    return cleaned_lines


def process_hashtags(lines, remove=False, heuristic=True):
    
    cleaned_lines = []
    for line in lines:
        words = re.split('(#\w+)', line)
        for i, word in enumerate(words):
            if len(word) == 0 or word[0] != '#':
                continue
            
            if heuristic and re.match("^#(([A-Z]+)|([a-zA-Z][a-z]+))$", word):
                words[i] = word[1:]
            elif remove:
                words[i] = ''
            else:
                hashtag_hook(word)

        cleaned_lines.append(''.join(words))
    return cleaned_lines

def process_emojis(lines, remove=False):
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    
    process = '' if remove else emoji_hook
    
    cleaned_lines = []
    for line in lines:
        cleaned_line = emoji_pattern.sub(process, line)
        cleaned_lines.append(cleaned_line)
    return cleaned_lines

def case_hook(string):
    return string.group(0) if len(string.group(0)) == 1 else string.group(0).lower()

def process_case(lines):
    cleaned_lines = []
    for line in lines:
        line = re.sub(r"[A-Z']+", case_hook, line, re.UNICODE)
        cleaned_lines.append(line)
    return cleaned_lines

def process_case(lines):
    cleaned_lines = []
    for line in lines:
        cleaned_lines.append(line.lower())
    return cleaned_lines

def process_contractions(lines):
    cleaned_lines = []
    for line in lines:
        # From stackoverflow
        # https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/19794953#19794953
        line = re.sub(r"won't", "will not", line, re.UNICODE)
        line = re.sub(r"can\'t", "can not", line, re.UNICODE)

        # general
        line = re.sub(r"\'ve", " have", line, re.UNICODE)
        line = re.sub(r"n\'t", " not", line, re.UNICODE)
        line = re.sub(r"\'re", " are", line, re.UNICODE)
#         line = re.sub(r"\'s", " is", line, re.UNICODE) # This one is 
        line = re.sub(r"\'d", " would", line, re.UNICODE)
        line = re.sub(r"\'ll", " will", line, re.UNICODE)
        line = re.sub(r"\'t", " not", line, re.UNICODE)
        line = re.sub(r"\'m", " am", line, re.UNICODE)
        cleaned_lines.append(line)
    return cleaned_lines

def clean_non_ascii(lines):
    
    contraction_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    
    cleaned_lines = []
    for line in lines:
        cleaned_line = line.encode('ascii',errors='ignore')
        cleaned_lines.append(cleaned_line)
    return cleaned_lines

import csv

def normalization_dictionaries(lines, dictionaries):

    token_lines = []
    for line in lines:
        token_lines.append(re.split("([\w']+)", line))
    
    for dictionary in dictionaries:
        dic = {}
        with open(dictionary, 'r') as file:
            reader = csv.reader(file, delimiter='|')
            for row in reader:
                dic[row[0].strip()] = row[1].strip()

        for tokens in token_lines:
            for i, token in enumerate(tokens):
                if token in dic:
                    tokens[i] = dic[token]
    #                 print token


    cleaned_lines = []
    for tokens in token_lines:
        cleaned_lines.append(''.join(tokens))

    return cleaned_lines
    

In [71]:
class Tree:
    
    def __init__(self):
        self.left = None
        self.right = None
        self.data = None
        self.letter = None

# Constructs a greedy tree
def construct_tree_rec(U, ids, alphamap, words):
    if U.shape[0] < 20 or U.shape[1] <= 1:
        T = Tree()
        T.data = ids
        return T
    
    bestd = np.inf
    bests = 0
    besti = 0
    
    for i, u in enumerate(U.T):
        s = int(np.sum(u))
        d = np.abs(len(U) / 2 - s)
        if (d < bestd):
            bests = s
            bestd = d
            besti = i

    T = Tree()
    T.letter = alphamap[besti]          

    reorder = np.argsort(U.T[besti])
    sep = len(U) - bests
    
    alphamap = np.delete(alphamap, besti)
    U = np.delete(U[reorder], besti, axis=1)
    ids = ids[reorder]
    
    Ul = U[:sep]
    Ur = U[sep:]
    idsl = ids[:sep]
    idsr = ids[sep:]

    if (len(idsl) > 0): T.left = construct_tree_rec(Ul, idsl, alphamap, words)
    if (len(idsr) > 0): T.right = construct_tree_rec(Ur, idsr, alphamap, words)
    return T


def candidate_edit_dist(T, word, edit=2):
    
    if edit < 0:
        return np.array([], dtype=int)
    
    if T.left is None or T.right is None:
        return T.data if T.data is not None else np.array([], dtype=int)
    
    if T.letter in word:
        left = candidate_edit_dist(T.left, word, edit-1)
        right = candidate_edit_dist(T.right, word, edit)
    else:
        left = candidate_edit_dist(T.left, word, edit)
        right = candidate_edit_dist(T.right, word, edit-1)
    
    return np.hstack((left, right))

def candidate_edit_dist(T, word, edit=2):
    
    if edit < 0:
        return np.array([], dtype=int)
    
    if T.left is None and T.right is None:
        return T.data if T.data is not None else np.array([], dtype=int)
    
    if T.left is None:
        left = np.array([], dtype=int)
    elif T.letter in word:
        left = candidate_edit_dist(T.left, word, edit-1)
    else:
        left = candidate_edit_dist(T.left, word, edit)
        
    if T.right is None:
        right = np.array([], dtype=int)
    elif T.letter in word:
        right = candidate_edit_dist(T.right, word, edit)
    else:
        right = candidate_edit_dist(T.right, word, edit-1)
    
    return np.hstack((left, right))

def clean_dictionary(words):
    
    # Clean dictionary
    cleaned_words = []
    
    dic = {}
    for word in nltk.corpus.words.words():
        dic[word] = 1
    
    for word in words:
        
        word = word.lower()
        if re.match('^\w+$', word) is None:
            continue
            
        if word not in dic:
            continue
            
        cleaned_words.append(word)
            
    print("removed : %d" % (len(words) - len(cleaned_words)))

    return np.array(cleaned_words)

def construct_tree(cleaned_words):
    
    cleaned_words = np.array(cleaned_words)

    alphabet = {key: 1 for key in ''.join(cleaned_words)}
    alphamap = np.empty(len(alphabet), dtype='|S1')
    for i, key in enumerate(alphabet.keys()):
        alphabet[key] = i
        alphamap[i] = key
    
    U = np.empty((len(cleaned_words), len(alphabet)))
    for id, word in enumerate(cleaned_words):
        vec = np.zeros(len(alphabet))
        for l in word:
            vec[alphabet[l]] = 1
        
        U[id] = vec
    
    ids = np.arange(len(cleaned_words), dtype=int)
    T = construct_tree_rec(U, ids, alphamap, cleaned_words)
    return T

In [72]:
# This algorithm has been adapted from the pseudo-code given
# by https://en.wikipedia.org/wiki/Levenshtein_distance
class Levenshtein:
    
    def __init__(self):
        self.m = 1
        self.n = 1
        self.init_dist()
    
    def init_dist(self):
        self.d = np.empty((self.m + 1, self.n + 1), dtype=int)
        self.d[0, 0] = 0
        self.d[1:, 0] = np.arange(self.m) + 1
        self.d[0, 1:] = np.arange(self.n) + 1
    
    def dist(self, s, t):
    
        m, n = len(s), len(t)
    
        # Reallocate and initialize only if necessary
        if m > self.m or n > self.n:
            self.m = max(self.m, m)
            self.n = max(self.n, n)
            self.init_dist()

        for j in range(n):
            for i in range(m):
                cost = 0 if t[j] == s[i] else 1
                self.d[i+1, j+1] = min(self.d[i, j+1] + 1,  # deletion
                                       self.d[i+1, j] + 1,  # insertion
                                       self.d[i, j] + cost) # substitution
        return self.d[m, n]
    
class DamerauLevenshtein:
    
    
    def __init__(self):
        self.m = 1
        self.n = 1
        self.init_dist()
    
    def init_dist(self):
        self.d = np.empty((self.m + 1, self.n + 1), dtype=int)
        self.d[0, 0] = 0
        self.d[1:, 0] = np.arange(self.m) + 1
        self.d[0, 1:] = np.arange(self.n) + 1
    
    def dist(self, s, t):
    
        m, n = len(s), len(t)
    
        # Reallocate and initialize only if necessary
        if m > self.m or n > self.n:
            self.m = max(self.m, m)
            self.n = max(self.n, n)
            self.init_dist()

        for j in range(n):
            for i in range(m):
                cost = 0 if t[j] == s[i] else 1
                self.d[i+1, j+1] = min(self.d[i, j+1] + 1,  # deletion
                                       self.d[i+1, j] + 1,  # insertion
                                       self.d[i, j] + cost) # substitution
                if i > 0 and j > 0 and s[i] == t[j-1] and s[i-1] == t[j]:
                    self.d[i+1, j+1] = min(self.d[i+1, j+1],
                                           self.d[i-1, j-1] + cost)
        return self.d[m, n]


In [73]:
import nltk

def tokenize(tweets):
    token_tweets = []
    for tweet in tweets:
        token_tweets.append(nltk.word_tokenize(tweet))
    return token_tweets

def pos_tagging(token_tweets):
    tags = []
    for tokens in token_tweets:
        tagged = nltk.pos_tag(tokens)
        tags.append([t[1] for t in tagged])
    return tags

In [59]:
def process_context_similarity(token_tweets, tweets_tags):

    edit_dist = DamerauLevenshtein()

    aff_edit = [1, 0.3, 0.1]

    tokens_list = []

    for token_list, tag_list in zip(token_tweets, tweets_tags):
        processed_tokens = []
        for j, (word, tag) in enumerate(zip(token_list, tag_list)):

            word = word.lower()

            if tag == 'NNP' or len(tag) < 2:
                processed_tokens.append(word)
                continue


            contextv = model.context2vec(token_list, j)
            contextv = np.dot(w, contextv)

            ids = candidate_edit_dist(T, word)
            candidates = cleaned_words[ids]


            best_aff = 0
            best_candidate = ''
            for candidate in candidates:

                ed = edit_dist.dist(word, candidate)
                if ed > 2: continue

                target = aff_edit[ed]
                context = contextv[word2index[candidate]]

                aff = context * target
                if aff > best_aff:
                    best_aff = aff
                    best_candidate = candidate           

            if best_aff == 0 or best_aff < 0.1:
                best_candidate = word
    #         else:
    #             print word + " -> " + best_candidate + " " + str(best_aff)

            processed_tokens.append(best_candidate)

        tokens_list.append(processed_tokens)

    return tokens_list

In [60]:
from context2vec.common.model_reader import ModelReader

model_param_file = "data/context2vec.ukwac.model.params"

model_reader = ModelReader(model_param_file)
w = model_reader.w
word2index = model_reader.word2index
index2word = model_reader.index2word
model = model_reader.model

Reading config file: data/context2vec.ukwac.model.params
Config:  {'config_path': 'data/', 'model_file': 'context2vec.ukwac.model', 'deep': 'yes', 'drop_ratio': '0.0', 'words_file': 'context2vec.ukwac.words.targets', 'unit': '300'}


In [61]:
cleaned_words = clean_dictionary(model_reader.index2word)
T = construct_tree(cleaned_words)

100%|██████████| 160562/160562 [00:00<00:00, 780787.76it/s]


removed : 122297


In [95]:
tweets = ["@RT_com: #Paris txi drivers turned off their meters, took people home for free - reports https://t.co/UUjfMTCXsM https://t.co/1TicKMbNJy",
         "I've no idea were you could find this"]

In [96]:
tweets_before = tweets

In [97]:
tweets = process_links(tweets, remove=True)
tweets = process_emojis(tweets, remove=True)
tweets = process_names(tweets, remove=True)
tweets = process_hashtags(tweets, remove=True)
tweets = clean_non_ascii(tweets)
tweets = process_case(tweets)
# Adapted from http://www.smart-words.org/abbreviations/text.html
# and http://www.hlt.utdallas.edu/~yangl/data/Text_Norm_Data_Release_Fei_Liu/
tweets = normalization_dictionaries(tweets, ['data/Test_Set_3802_Pairs.txt', 'data/short_abbrev_list.txt'])
tweets = process_contractions(tweets)

token_tweets = tokenize(tweets)
tweets_tags = pos_tagging(token_tweets)

final_tokens = process_context_similarity(token_tweets, tweets_tags)

In [98]:
for a, b in zip(final_tokens, tweets_before):
    print(b)
    print(' '.join(a))
    print("---")

@RT_com: #Paris txi drivers turned off their meters, took people home for free - reports https://t.co/UUjfMTCXsM https://t.co/1TicKMbNJy
paris taxi drivers turned off their meters , took people home for free - reports
---
I've no idea were you could find this
i have no idea where you could find this
---
